<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tools" data-toc-modified-id="Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tools</a></span></li><li><span><a href="#path" data-toc-modified-id="path-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>path</a></span><ul class="toc-item"><li><span><a href="#string-and-unicode" data-toc-modified-id="string-and-unicode-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>string and unicode</a></span></li><li><span><a href="#path()" data-toc-modified-id="path()-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>path()</a></span></li><li><span><a href="#string()" data-toc-modified-id="string()-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>string()</a></span></li></ul></li><li><span><a href="#Path-decomposition" data-toc-modified-id="Path-decomposition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Path decomposition</a></span></li><li><span><a href="#directory_entry" data-toc-modified-id="directory_entry-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>directory_entry</a></span></li><li><span><a href="#directory_iterator" data-toc-modified-id="directory_iterator-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>directory_iterator</a></span></li><li><span><a href="#Non-member-function" data-toc-modified-id="Non-member-function-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Non-member function</a></span></li></ul></div>

# Tools

```bash
sudo add-apt-repository ppa:jonathonf/gcc-9.0
sudo apt-get install gcc-9 g++-9
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 900 --slave /usr/bin/g++ g++ /usr/bin/g++-9
```

In [214]:
%%bash
g++ --version | grep g++

g++ (Ubuntu 9-20190428-1ubuntu1~18.04.york0) 9.0.1 20190428 (prerelease) [gcc-9-branch revision 270630]


In [215]:
%%file cxx.cxx

#include <iostream>
#include <fstream>
#include <string>
#include <stdlib.h>

bool starts_with(const std::string& str, const std::string& sub) {
    return sub.length() <= str.length()
        && equal(sub.begin(), sub.end(), str.begin());
}

int main(int argc, char const *argv[]) {
    using namespace std;
    
    std::string cmd = "g++";
    cmd.append(" test.cxx");
    
    for(int a = 1; a < argc; a++) {
        std::string arg = argv[a];
        if(starts_with(arg, "-std=")) {
            cmd.append(" ").append(arg);
        } else if(starts_with(arg, "-l")) {
            cmd.append(" ").append(arg);
        } else {
            cout << "unkownn arguments: " << arg << endl;
            return 1;
        }
    }
    
    
    ofstream of("test.cxx");
    for(;cin;) {
        std::string line;
        getline(cin, line);
        of << line << endl;
    }
    return system((cmd + " -o test && ./test").c_str());
}

Overwriting cxx.cxx


In [216]:
%%script bash
g++ -std=c++03 cxx.cxx -o cxx

In [217]:
%%file demangle.h

#include <string>
#include <cxxabi.h>
#include <stdlib.h>

inline std::string demangle(char const *name) {
    int status;
    char * realname = abi::__cxa_demangle(name,0,0, &status);
    std::string realname_ = realname;
    free(realname);
    return realname_;    
}

Overwriting demangle.h


# path

## string and unicode

In [223]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    const char *ptr = "\U0001F60A";
    cout << ptr << endl;

    // g++ version <= 8, auto -> char const *
    // g++ version >= 9, auto -> char8_t const *
    auto u8_ptr = u8"\U0001F60A";
    cout << demangle(typeid(u8_ptr).name()) << endl;
    // you would see content of string, for g++ version <= 8
    // you would see a pointer address, for g++ version >= 8
    cout << u8_ptr << endl;

    const char16_t *u16_ptr = u"\U0001F60A";
    cout << u16_ptr << endl;
    
    const char32_t *u32_ptr = U"\U0001F60A";
    cout << u32_ptr << endl;

    std::string str = ptr;
    std::u8string u8_str = u8_ptr;
    std::u16string u16_str = u16_ptr;
    std::u32string u32_str = u32_ptr;
     
    cout << str << endl;
    
    // can't use the following, we have no u8cout, u16cout, u32cout
    // cout << u8_str << endl;
    // cout << u16_str << endl;
    // cout << u32_str << endl;
}


😊
char8_t const*
0x562e09dab27c
0x562e09dab282
0x562e09dab288
😊


## path()

In [251]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    fs::path path1 { "\U0001F60A" };
    fs::path path2 { u8"\U0001F60A" };
    
    // g++ refuse to convert unicode for UTF-16
    fs::path path4 { u"ascii" };
    // fs::path path4 { u"\U0001F60A" };
    fs::path path5 { U"\U0001F60A" };
}


## string()

In [229]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    fs::path path { "\U0001F60A" };
    cout << path << endl;
    cout << path.string() << endl;
    
    // If gcc version <= 8, u8string() will return string
    // This will change in the future, u8string() will return u8string
    // cout << path.u8string() << endl;
    
    // not working, we have no u16cout, u32cout
    // cout << path.u16string() << endl;
    // cout << path.u32string() << endl;
}

"😊"
😊


# Path decomposition

In [230]:
%%script ./cxx -std=c++17 -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    auto path = fs::current_path();
    path.append("a.b");
    cout <<  path << endl;
    cout <<  path.root_name() << endl;
    cout <<  path.root_path() << endl;
    cout <<  path.relative_path() << endl;
    cout <<  path.parent_path() << endl;
    cout <<  path.filename() << endl;
    cout <<  path.stem() << endl;
    cout <<  path.extension() << endl;
        
}


"/home/liangh/projects/blog/JupyterNotebooks/a.b"
""
"/"
"home/liangh/projects/blog/JupyterNotebooks/a.b"
"/home/liangh/projects/blog/JupyterNotebooks"
"a.b"
"a"
".b"


# directory_entry

In [231]:
%%script ./cxx -std=c++17 -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    auto path = fs::current_path();
    fs::directory_entry de { path };
    cout << de.exists() << endl;
    cout << de.is_directory() << endl;
    cout << de.is_regular_file() << endl;
    cout << de.is_symlink() << endl;
    try {
        cout << de.file_size() << endl;        
    } catch(...) {        
    }
        
}


1
1
0
0


# directory_iterator

In [79]:
%%script ./cxx -std=c++17 -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    auto path = fs::current_path();
    fs::directory_iterator di { path };
    for(int i = 0; di != fs::directory_iterator() && i < 2; ++di, ++i) {
        cout << (*di).path().filename() << endl;
    }
 
    auto i = 0;
    for(auto const &de : fs::directory_iterator(path)) {
        // de: const &directory_entry
        cout << de.path().filename() << endl;

        ++i;
        if(i >=2) break;
    }
}


"nullptr"
"structure_binding_tuple.cxx"
"nullptr"
"structure_binding_tuple.cxx"


# Non-member function

In [298]:
%%script ./cxx -std=c++17 -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    cout << fs::current_path() << endl;
    cout << fs::space(fs::current_path()).available/(1024*1024*1024) << "T" << endl;
    
    fs::path path { "." };
    auto parent_path = fs::canonical(path).parent_path();
    cout << fs::absolute(path) << endl;
    cout << fs::canonical(path) << endl;
    cout << fs::weakly_canonical(path) << endl;
    cout << (parent_path) << endl;
    cout << fs::relative(parent_path) << endl;
    
    system("rm b -rf");
    fs::create_directory(fs::path{"a"});
    fs::create_directory(fs::path{"a/1"});
    fs::copy(fs::path{"a"}, fs::path{"b"}, fs::copy_options::recursive);
    cout << exists(fs::path{"b/1"}) << endl;
    fs::rename(fs::path{"b/1"}, fs::path{"b/2"});
    fs::remove(fs::path{"b/2"});
    cout << exists(fs::path{"b/1"}) << exists(fs::path{"b/2"}) << endl;
    
    cout << fs::is_directory(fs::current_path()) << endl;
    
    
}


"/home/liangh/projects/blog/JupyterNotebooks"
1875T
"/home/liangh/projects/blog/JupyterNotebooks/."
"/home/liangh/projects/blog/JupyterNotebooks"
"/home/liangh/projects/blog/JupyterNotebooks"
"/home/liangh/projects/blog"
".."
1
00
1
